In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [10]:
df_ratings = pd.read_csv('/ratings.csv')#평점파일 업로드
rating_data = pd.read_csv('/ratings.csv') 
df_movies =  pd.read_csv('/movies.csv') #영화파일 업로드
rating_data.head() #평점 데이터 구조 보기

,userId,movieId,rating,timestamp
0,1,110,1.0,1.425942e+09
1,1,147,4.5,1.425942e+09
2,1,858,5.0,1.425942e+09
3,1,1221,5.0,1.425942e+09
4,1,1246,5.0,1.425942e+09


In [36]:
df_user_movie_ratings.shape

(11729, 20806)

In [18]:
df_user_movie_ratings = df_ratings.pivot(index='userId',columns='movieId',values='rating').fillna(0)

In [24]:
df_user_movie_ratings.head()
#type(df_user_movie_ratings) -> 판다스 데이터프레임의 파일형식(피봇테이블프레임)

movieId,1,2,3,4,5,6,7,8,9,10,...,175773,175775,175777,175779,175945,175975,176165,176211,176219,176271
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##여기까지는 특정영화 상관계수 분석과 동일
###바꿀파트
pivot table을 matrix로 변환
np.mean(axis = 1)을 통해 각 사용자들이 매기는 평점 평균을 구함
1에서 구한 값과 2에서 구한 값을 빼서 사용자-평균 데이터 값을 변경

In [27]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = np.asmatrix(df_user_movie_ratings)

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)#axis = 1 가로로 평균함수 적용을 의미 (사용자가 내린 평점들의 평균)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1) #reshape(-1,1)은 열을 1로 했을 때 -1은 행의 갯수를 자동으로 지정해줌

In [34]:
matrix

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [4., 0., 3., ..., 0., 0., 0.]])

In [35]:
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175773,175775,175777,175779,175945,175975,176165,176211,176219,176271
0,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,...,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551,-0.005551
1,-0.003509,-0.003509,-0.003509,-0.003509,2.996491,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509,...,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509,-0.003509
2,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,...,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490,-0.001490
3,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,...,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430,-0.010430
4,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,2.994665,-0.005335,-0.005335,-0.005335,...,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335,-0.005335


여기까지 진행하면 초기에 만들었던 user-movie pivot table 값이 matrix_user_mean 변경되었습니다.

즉, 아래와 같이 변경된 것이죠.

1. 사용자 11729명이 20806개의 영화에 대해 평점을 매긴 값이 존재
2. 사용자 11729명의 각각 평균 평점을 구해서
3. 사용자 11729명의 20806개의 영화에 대해 평점을 조금 변경 -> 1에서 구한 값 - 2에서 구한 값 -> (+)의 값이면 사용자가 준 평균 평점보다 높음, (-)의 값이면 사용자가 준 평균 평점보다 낮음

In [37]:
U, sigma, Vt = svds(matrix_user_mean, k = 12) #상위 12개로 압축

In [38]:
print(U.shape)
print(sigma.shape)
#현재 이 Sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태입니다.
#즉, 0이 포함된 대칭행렬로 변환할 때는 numpy의 diag를 이용해야 합니다
print(Vt.shape)

(11729, 12)
(12,)
(12, 20806)


In [40]:
sigma = np.diag(sigma)

In [42]:
sigma.shape

(12, 12)

원본 user-movie 평점 행렬이 있었음


이를 user의 평균 점수를 빼서 matrix_user_mean 이라는 행렬로 만듬


2번의 값을 SVD를 적용해 U, Sigma, Vt 행렬을 구했음


Sigma 행렬은 현재 0이 포함이 되지 않은 값으로만 구성되어 있음. 이를 대칭행렬로 변환


자! 이제 여기서 matrix_user_mean을 SVD를 적용해 분해를 한 상태입니다.
이제, 다시 원본 행렬로 복구시켜야겠죠?

원본 행렬로 복구시키는 방법은 아래와 같습니다.

U, Sigma, Vt의 내적을 수행
즉, np.dot(np.dot(U, sigma), Vt)를 수행하면 됩니다.

그리고 아까 사용자 평균을 빼주었으니 여기서는 더해줍니다.

In [43]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [44]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175773,175775,175777,175779,175945,175975,176165,176211,176219,176271
0,0.371518,-0.018959,-0.053955,0.004128,-0.038014,-0.027068,-0.036580,-0.006025,-0.026025,-0.247463,...,-0.001346,-0.001346,-0.001346,-0.001346,-0.001242,-0.001015,-0.000526,-0.000817,-0.001325,-0.001147
1,1.288428,0.225629,0.692601,0.056374,0.630476,0.870206,0.738344,0.031956,0.268481,0.201601,...,0.002667,0.002667,0.002667,0.002667,0.001249,0.000859,0.000284,0.001121,0.001400,0.001294
2,0.280772,0.073842,0.030339,-0.001302,0.038421,-0.000768,0.028459,0.009704,-0.003927,0.016786,...,0.001969,0.001969,0.001969,0.001969,0.001112,0.002309,0.003094,0.002591,0.002357,0.002259
3,0.363823,0.081465,0.014698,0.012425,0.039112,0.037463,0.046895,0.005437,-0.034019,-0.132706,...,-0.000176,-0.000176,-0.000176,-0.000176,-0.001382,0.000563,0.001552,-0.000052,-0.000120,-0.001050
4,0.524745,0.013007,0.036237,-0.043122,0.011428,0.090777,0.019721,-0.004304,-0.005995,-0.067824,...,0.001642,0.001642,0.001642,0.001642,0.001207,0.002229,0.002640,0.003091,0.002750,0.003638


In [45]:
df_svd_preds.shape

(11729, 20806)

함수를 하나 만듭니다. 이 함수의 기능은 아래와 같습니다.

인자로 사용자 아이디, 영화 정보 테이블, 평점 테이블 등을 받음

사용자 아이디에 SVD로 나온 결과의 영화 평점이 가장 높은 데이터 순으로 정렬

사용자가 본 데이터를 제외

사용자가 안 본 영화에서 평점이 높은 것을 추천

In [46]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id - 1 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됨.
    # 나의 예상 평점들을 높은 순으로 보여줌
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
    #실제 내린 평점 데이터프레임에서 내 유저아이디에 해당하는 열(내가 평점을 내린 영화들) 저장
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합치고 #평점 값이 높은 순으로 정렬
    user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]

    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [47]:
already_rated, predictions = recommend_movies(df_svd_preds, 330, df_movies, df_ratings, 10)

In [50]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,genres
232,330,8961,5.0,1.257630e+09,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy
14,330,296,5.0,1.290916e+09,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
67,330,1302,5.0,1.257639e+09,Field of Dreams (1989),Children|Drama|Fantasy
139,330,3868,5.0,1.257639e+09,"Naked Gun: From the Files of Police Squad!, Th...",Action|Comedy|Crime|Romance
26,330,551,5.0,1.257639e+09,"Nightmare Before Christmas, The (1993)",Animation|Children|Fantasy|Musical
297,330,55820,5.0,1.290916e+09,No Country for Old Men (2007),Crime|Drama
22,330,441,5.0,1.290913e+09,Dazed and Confused (1993),Comedy
305,330,58559,5.0,1.257622e+09,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
306,330,58998,5.0,1.257639e+09,Forgetting Sarah Marshall (2008),Comedy|Romance
116,330,2792,5.0,1.257622e+09,Airplane II: The Sequel (1982),Comedy


In [51]:
predictions

,movieId,title,genres,Predictions
3684,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,3.411818
1255,1682,"Truman Show, The (1998)",Comedy|Drama|Sci-Fi,2.852503
975,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure,2.747181
3047,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",Action|Drama|Romance,2.685172
459,541,Blade Runner (1982),Action|Sci-Fi|Thriller,2.574487
929,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller,2.570951
4555,6711,Lost in Translation (2003),Comedy|Drama|Romance,2.542814
25,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,2.473328
604,778,Trainspotting (1996),Comedy|Crime|Drama,2.296393
41,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,2.269572
